In [5]:
agent_dir = r"C:\Users\TOMI\Documents\customer_personality_profiling\data\raw\agent"
caller_dir = r"C:\Users\TOMI\Documents\customer_personality_profiling\data\raw\caller"
output_dir = r"C:\Users\TOMI\Documents\customer_personality_profiling\data\merged"

os.makedirs(output_dir, exist_ok=True)

In [6]:
import librosa
import soundfile as sf
import os

In [7]:
from pydub import AudioSegment

def merge_audio(agent_file, caller_file, output_path):
    # Load and normalize audio
    agent_audio = AudioSegment.from_file(agent_file).set_frame_rate(16000).set_channels(1)
    caller_audio = AudioSegment.from_file(caller_file).set_frame_rate(16000).set_channels(1)

    # Convert to raw samples
    agent_samples = agent_audio.get_array_of_samples()
    caller_samples = caller_audio.get_array_of_samples()

    # Fix sample-level mismatch
    if len(agent_samples) < len(caller_samples):
        diff_samples = len(caller_samples) - len(agent_samples)
        silence_duration = (diff_samples / 16000) * 1000  # convert samples to ms
        agent_audio += AudioSegment.silent(duration=silence_duration)
    elif len(caller_samples) < len(agent_samples):
        diff_samples = len(agent_samples) - len(caller_samples)
        silence_duration = (diff_samples / 16000) * 1000
        caller_audio += AudioSegment.silent(duration=silence_duration)

    # Re-check and trim if still off by 1 sample
    if len(agent_audio.get_array_of_samples()) != len(caller_audio.get_array_of_samples()):
        min_samples = min(len(agent_audio.get_array_of_samples()), len(caller_audio.get_array_of_samples()))
        agent_audio = agent_audio[:min_samples]
        caller_audio = caller_audio[:min_samples]

    # Merge into stereo (agent = left, caller = right)
    merged = AudioSegment.from_mono_audiosegments(agent_audio, caller_audio)

    # Export merged file
    merged.export(output_path, format="wav")
    print(f"Merged: {output_path}")


In [3]:
from pydub import AudioSegment

In [8]:
def merge_audio(agent_file, caller_file, output_path):
    agent_audio = AudioSegment.from_file(agent_file)
    caller_audio = AudioSegment.from_file(caller_file)

    agent_audio =agent_audio.set_frame_rate(16000).set_channels(1)
    caller_audio = caller_audio.set_frame_rate(16000).set_channels(1)

    if len(agent_audio) > len(caller_audio):
        caller_audio = caller_audio + AudioSegment.silent(duration=len(agent_audio) - len(caller_audio))
    else:
        agent_audio = agent_audio + AudioSegment.silent(duration=len(caller_audio) - len(agent_audio))

    merged = AudioSegment.from_mono_audiosegments(agent_audio, caller_audio)

    merged.export(output_path, format="wav")
    print(f"Merged: {output_path}")

In [9]:
import os

In [14]:
for agent_file in os.listdir(agent_dir):
    if agent_file.endswith(".wav"):
        caller_file = f"{agent_file}_caller.wav"
        agent_path = os.path.join(agent_dir, agent_file)
        caller_path = os.path.join(caller_dir, agent_file)

        if os.path.exists(caller_path):
            output_path = os.path.join(output_dir, agent_file.replace(".wav", "_merged.wav"))
            merge_audio(agent_path, caller_path, output_path)
        else:
            print(f"Caller file not found for {agent_file}")


ValueError: attempt to assign array of size 817758 to extended slice of size 817759